<a href="https://colab.research.google.com/github/Verose/ML_Applications_TAU/blob/master/Assignment_1_basic_statistics_and_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

# Section 1: Data Science

In [0]:
# TODO: Load the flights training data into 'data'

#drive.mount('/content/drive')


In [0]:
# TODO: How many flights got delayed at Charlotte?

# CLT_flights = data[data['Origin'] == "CLT"]

#...

In [0]:
# TODO: Show the number of departures per hour-of-day

# You are adviced to use 'groupby'. More info can be found here:
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html


In [0]:
# TODO: Plot number of departures (Y axis) per hour-of-day (X axis)


In [0]:
# TODO: Plot the average number of delays per hour-of-day.


In [0]:
# TODO: Find the worse pair of origin and carrier 

# You may want to use 'idxmax' (like argmax, returning the row with max value of specific column)


# Section 2: Supervised Learning
Here, you are requested to use all features to predict the response parameter dep_delayed_15min

In [0]:
# TODO: encode all categorical features as one-hot representation

# Month	DayofMonth	DayOfWeek	DepTime	UniqueCarrier	Origin	Dest	Distance	dep_delayed_15min


In [0]:
# TODO: Split the data to train and test. Use test_size=0.3, and random_state=1 



In [0]:
# TODO: Fit a LogisticRegression classifier to the training set


In [0]:
# TODO: print confusion matrix over the testing set


In [0]:
# TODO: Calculate Precision, Recall and F score

In [0]:
# TODO: Similarly, print precision/recall for the following classifiers:

# sklearn.naive_bayes.BernoulliNB
# sklearn.neighbors.KNeighborsClassifier
# sklearn.svm.LinearSVC
